In [2]:
import requests
import pandas as pd
import json

![Params](nmdc_api_params.png)

GET
/dataset/{startyear}/{cruiseid}/{platform}

In [3]:
def fetch_sensordata(startyear, cruiseid, outname):
    api_url = 'http://underway-api.hi.no/apis/nmdapi/underway/v1'
    r = requests.get(f'{api_url}/dataset/{startyear}/{cruiseid}/9998', headers={"accept":"application/json"})
    with open(f'{outname}.json', 'w', encoding='utf-8') as f:
        json.dump(r.json(), f, ensure_ascii=False, indent=4)

In [5]:
fetch_sensordata(2021, 2021053, 'leg3')

In [6]:
def prep_and_separate_sensorjson(path, leg):
    f = open(path)

    data = json.load(f)

    sensors = data['sensor']
    for sensor in sensors:
        unit = sensor['unit']
        metadata = sensor['custommetadata']
        displayname = metadata[4]
        name = displayname['customvalue']
        data = sensor['timeseriesdata']

        data = { 'unit' : unit, 'sensor' : name, 'data': data }
        json.dumps(data)

        with open(f'{leg}/{name}.json', 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

In [8]:
prep_and_separate_sensorjson('leg2.json', 'Leg2')

## Prepare dataframes

In [21]:
def prep_df(file, prefix):
    f = open(file)
    data = json.load(f)
    name = data['sensor']
    df = pd.json_normalize(data['data'])
    df.dataid = pd.to_datetime(df.dataid, format='%Y-%m-%d %H:%M:%S')
    df = df.set_index('dataid')
    df = df.drop(columns=['sensortimestamp', 'geolocation'])
    df = df.rename(columns={'sensorvalue':name.split(prefix)[1]})
    
    return df


rel_sensors = ['FerryBox.C3_CDOMFluorescence.json', 'FerryBox.C3_Temperature.json', 'FerryBox.C3_CHLAFluorescence.json',
               'FerryBox.C3_Turbidity.json','FerryBox.Optode_Saturation.json', 'FerryBox.SBE45_Salinity.json',
               'NMEA.Trykk.json', 'NMEA.Humidity.json']

append_str = 'BlueInsight_'
blueinsight_sensors = [append_str + sub for sub in rel_sensors]

def combine_sensordfs(sensors, leg, prefix):
    prev_df = pd.DataFrame(columns=['dataid']) #empty dataframe with column dataid
    for sensor in sensors:
        df = prep_df(f'{leg}/{sensor}', prefix)
        new_df = pd.merge(df,prev_df,on='dataid', how="outer")
        prev_df = new_df

    return new_df
    

In [27]:
df = combine_sensordfs(blueinsight_sensors, 'Leg4', 'BlueInsight_')
df.to_csv('leg4_prepped.csv', index = False)